<a href="https://colab.research.google.com/github/purbid/jre_chatbot_transformer/blob/main/jre_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This Notebook has the code to build a chatbot, using transformers. 
### The chatbot is trained on conversations from The Joe Rogan Experience.

In [1]:
!pip install torchtext==0.6.0
!pip install torch==1.8.0


     |████████████████████████████████| 71kB 2.9MB/s 
     |████████████████████████████████| 1.2MB 5.4MB/s 
  Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
     |████████████████████████████████| 735.5MB 24kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102


In [2]:
from os import listdir
from os.path import isfile, join
import torch.nn as nn
from torch import Tensor
onlyfiles = [f for f in listdir("./drive/MyDrive/jre_transcripts") if isfile(join("./drive/MyDrive/jre_transcripts", f))]

In [3]:
pocast_transcript=[]
data=[]
for itr,i in enumerate(onlyfiles):
  if itr%10==0:
    print(i)
  podcast_curr=[]
  with open("./drive/MyDrive/jre_transcripts/"+str(i)) as f:
    lines = f.readlines()
    for i in range(1,len(lines), 3):
        podcast_curr.append(lines[i].strip())
        if i%2==0:
          data.append([lines[i-3].strip(), lines[i].strip()])
    f.close()
  pocast_transcript.append(podcast_curr)

Elon Musk Talks About Colonizing the Galaxy-transcript.txt
#1558 - Tristan Harris-transcript.txt
#1547 - Colin Quinn-transcript.txt
#1537 - Lex Fridman-transcript.txt
#1525 - Tim Dillon-transcript.txt
#1468 - Alonzo Bodden-transcript.txt
#1456 - Michael Shermer-transcript.txt
#1445 - Andy Stumpf-transcript.txt
#1436 - Adam Curry-transcript.txt
JRE MMA Show #91 with Radio Rahim-transcript.txt
#1417 - Kevin Ross-transcript.txt
#1407 - Michael Malice-transcript.txt


In [4]:
import pandas as pd
df_chat=pd.DataFrame(data, columns=["dialogue","response"])

In [4]:
##### create vocab using torchtext
#### create using spacy

In [5]:
df_chat["src_len"]=df_chat.dialogue.str.split().str.len()
df_chat["tgt_len"]=df_chat.response.str.split().str.len()


In [42]:
df_chat = df_chat.query("src_len>=4").query("src_len<=200").query("tgt_len>=4").query("tgt_len<=200")

In [43]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df_chat, test_size=0.25)
train.to_csv("train_dialogue.csv", index=False)
val.to_csv("val_dialogue.csv", index=False)

In [44]:
# import de_core_news_sm
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# from torchtext.datasets import Multi30k
# from typing import Iterable, List


SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'en'

# token_transform = {}
# vocab_transform = {}



# token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
# token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
#     language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

#     for data_sample in data_iter:
#         print(token_transform[language](data_sample[language_index[language]]))

# UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
#     # Training data Iterator
#     # train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
#     # Create torchtext's Vocab object
#     vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train, ln),
#                                                     min_freq=1,
#                                                     specials=special_symbols,
#                                                     special_first=True)

# for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
#   vocab_transform[ln].set_default_index(UNK_IDX)

In [45]:
from torchtext import data
jre_text = data.Field(init_token = "<bos>", eos_token = "<eos>")
data_fields = [('dialogue', jre_text), ('response', jre_text)]
train,val = data.TabularDataset.splits(path='./', train='train_dialogue.csv', validation='val_dialogue.csv', format='csv', fields=data_fields)
print("building_vocab ...")
jre_text.build_vocab(train, max_size=30000, min_freq=2)
print("done building vocab ...")

building_vocab ...
done building vocab ...


In [46]:
vocab_transform={}
vocab_transform["en"]=jre_text.vocab

In [ ]:
x=dict(jre_text.vocab.freqs)
print(len(x))

jre_text.vocab.itos[26000]

In [ ]:
print(len(x.keys()))

In [27]:

import torch
from torch.nn import Transformer

batch_size=8
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train, val),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.dialogue),
    device=device,
)

In [13]:
# for x,y in enumerate(train_iterator):
#   print(y.dialogue.shape)
#   # l=x[1].permute(1,0)[0].tolist()
#   # m=[jre_text.vocab.itos[x1] for x1 in l]
#   # print(m)
#   # l=x[0].permute(1,0)[0].tolist()
#   # m=[jre_text.vocab.itos[x1] for x1 in l]
#   # print(m)
#   break

In [28]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [29]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [30]:
import torch
import math

torch.manual_seed(0)

SRC_VOCAB_SIZE = 30003
TGT_VOCAB_SIZE = 30003
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)
PAD_IDX=1
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [31]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
from torchtext.data import get_tokenizer
token_transform={}
token_transform["en"]=get_tokenizer("spacy")
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()

    losses = 0
    # train_dataloader = DataLoader(train_iterator, batch_size=BATCH_SIZE)
                                  # , collate_fn=collate_fn)

    for batch_idx, batch in enumerate(train_iterator):
        if batch_idx<20:
          continue
        src = batch.dialogue.to(device)
        tgt = batch.response.to(device)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        try:

          logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
          optimizer.zero_grad()
        except Exception as e:
          print(e)
          print("\n")
          print("src is on cuda: "+str(src))
          print("tgt_input is on cuda: "+str(tgt_input.shape))
          print("src_mask is on cuda: "+str(src_mask.shape))
          print("tgt_mask is on cuda: "+str(tgt_mask.shape))
          print("src_padding_mask is on cuda: "+str(src_padding_mask.shape))
          print("tgt_padding_mask is on cuda: "+str(tgt_padding_mask.shape))
          print("src_padding_mask is on cuda: "+str(src_padding_mask.shape))
          return src
          break
          

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
    return losses / len(train_dataloader)


def evaluate(model):
    model.eval()
    losses = 0
    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

In [32]:
device

device(type='cuda')

In [33]:

CUDA_LAUNCH_BLOCKING=1


In [35]:

from timeit import default_timer as timer
NUM_EPOCHS = 18
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    break
    end_time = timer()
    print("first epoch done")
    val_loss = evaluate(transformer)
    
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))



RuntimeError: ignored

In [22]:
train_loss.shape

torch.Size([1014, 8])